# Initial Loaders

In [1]:
import torch
import torch.nn as nn
from tqdm import tqdm
from time import time
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd

from torch.utils.data import DataLoader, Subset, random_split
from colorVideoDataset import ColorVideoDataset

In [2]:
DATASET = ColorVideoDataset('./colors', sequence_length=30)
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
train_dataset, test_dataset = random_split(DATASET, [int(0.8 * len(DATASET)), len(DATASET) - int(0.8 * len(DATASET))])
test_dataset, val_dataset = random_split(test_dataset, [int(0.5 * len(test_dataset)), len(test_dataset) - int(0.5 * len(test_dataset))])

# PyTorch 3D ResNet

In [8]:
from models.ThreeDResNet import get_3dResNet, get_resnet_transformer

c:\Users\Arnav Waghdhare\Desktop\Arnav20\Coding\Python\geture_research_ml\.venv\Lib\site-packages\torchvision\transforms\_functional_video.py:6: UserWarning: The 'torchvision.transforms._functional_video' module is deprecated since 0.12 and will be removed in the future. Please use the 'torchvision.transforms.functional' module instead.
  warnings.warn(
c:\Users\Arnav Waghdhare\Desktop\Arnav20\Coding\Python\geture_research_ml\.venv\Lib\site-packages\torchvision\transforms\_transforms_video.py:22: UserWarning: The 'torchvision.transforms._transforms_video' module is deprecated since 0.12 and will be removed in the future. Please use the 'torchvision.transforms' module instead.
  warnings.warn(


In [9]:
MODEL = get_3dResNet()

Using cache found in C:\Users\Arnav Waghdhare/.cache\torch\hub\facebookresearch_pytorchvideo_main


Net(
  (blocks): ModuleList(
    (0): ResNetBasicStem(
      (conv): Conv3d(3, 64, kernel_size=(1, 7, 7), stride=(1, 2, 2), padding=(0, 3, 3), bias=False)
      (norm): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (activation): ReLU()
      (pool): MaxPool3d(kernel_size=(1, 3, 3), stride=(1, 2, 2), padding=[0, 1, 1], dilation=1, ceil_mode=False)
    )
    (1): ResStage(
      (res_blocks): ModuleList(
        (0): ResBlock(
          (branch1_conv): Conv3d(64, 256, kernel_size=(1, 1, 1), stride=(np.int64(1), np.int64(1), np.int64(1)), bias=False)
          (branch1_norm): BatchNorm3d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (branch2): BottleneckBlock(
            (conv_a): Conv3d(64, 64, kernel_size=(1, 1, 1), stride=(1, 1, 1), bias=False)
            (norm_a): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (act_a): ReLU()
            (conv_b): Conv3d(64, 64, kern

Train, Test, Val Sets

## Dataloader and Training

In [10]:
def get_dataloader(dataset, subset_ratio : float | None = 0.1, batch_size : int = 2):
    transform = get_resnet_transformer()

    def collate_fn(batch):
        videos = []
        labels = []
        for video, label, _ in batch:
            video = video.permute(1, 0, 2, 3)
            video = transform({"video": video})["video"]
            videos.append(video)
            labels.append(torch.tensor(label, dtype=torch.long))
        
        videos = torch.stack(videos)
        labels = torch.stack(labels)
        return videos, labels
    
    if subset_ratio is not None:
        num_samples = int(len(dataset) * subset_ratio) 
        subset_indices = list(range(num_samples))
        subset = Subset(dataset, subset_indices)
        dataloader = DataLoader(subset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
    else:
        dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
    return dataloader

In [11]:
def infer_loop(model, dataloader, criterion = None) -> tuple:
    if criterion is None:
        criterion = nn.CrossEntropyLoss()
    
    running_loss = 0.0
    all_preds = []
    all_labels = []
    model.eval()
    for videos, labels in dataloader:
        videos, labels = videos.to(DEVICE), labels.to(DEVICE)

        outputs = model(videos)
        loss = criterion(outputs, labels)
        running_loss += loss.item()
        all_preds.extend(outputs.argmax(dim=1).cpu().detach().numpy())
        all_labels.extend(labels.cpu().detach().numpy())
        
    val_accuracy = accuracy_score(all_preds, all_labels)
    avg_loss = running_loss / len(dataloader)

    return (val_accuracy, avg_loss)

In [ ]:
def training_loop(model, train_dataloader, val_dataloader: DataLoader | None = None, 
                  epochs=5, learning_rate=2e-4, early_stopping_patience=10,
                  checkpoint_path='best_model_3DResNet.pth', lr_patience=2):

    results_dict = {
        'time_per_batch': [],
        'time_per_epoch': [],
        'train_accuracy': [],
        'train_loss': [],
        'val_accuracy': [],
        'val_loss': [],
        'precision': [],
        'recall': [],
        'f1': [],
        'learning_rate': [],
    }
    
    model = model.to(DEVICE)
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    
    # ReduceLROnPlateau - tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', patience=2)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode='min', factor=0.1, patience=lr_patience, verbose=True, min_lr=1e-7
    )
    
    # EarlyStopping - tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)
    best_val_loss = float('inf')
    epochs_without_improvement = 0
    
    epoch_pbar = tqdm(range(epochs), desc="Training Epochs")

    for epoch in epoch_pbar:
        # Training phase
        model.train()
        running_loss = 0.0
        epoch_start = time()
        all_preds = []
        all_labels = []
        
        for videos, labels in train_dataloader:
            videos, labels = videos.to(DEVICE), labels.to(DEVICE)

            optimizer.zero_grad()
            outputs = model(videos)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            all_preds.extend(outputs.argmax(dim=1).cpu().detach().numpy())
            all_labels.extend(labels.cpu().detach().numpy())
        
        epoch_time = time() - epoch_start
        results_dict['time_per_epoch'].append(epoch_time)
        
        # MetricsCallback - tracks precision, recall, f1
        results_dict['train_accuracy'].append(accuracy_score(all_labels, all_preds))
        results_dict['precision'].append(precision_score(all_labels, all_preds, average='weighted', zero_division=0))
        results_dict['recall'].append(recall_score(all_labels, all_preds, average='weighted', zero_division=0))
        results_dict['f1'].append(f1_score(all_labels, all_preds, average='weighted', zero_division=0))
        
        avg_train_loss = running_loss / len(train_dataloader)
        results_dict['train_loss'].append(avg_train_loss)
        
        val_accuracy, val_loss = infer_loop(model, val_dataloader, criterion)
        
        results_dict['val_accuracy'].append(val_accuracy)
        results_dict['val_loss'].append(val_loss)
        results_dict['learning_rate'].append(optimizer.param_groups[0]['lr'])
        
        # ReduceLROnPlateau callback
        scheduler.step(val_loss)
        
        # ModelCheckpoint - tf.keras.callbacks.ModelCheckpoint('best_model_TVN.h5', monitor='val_loss', save_best_only=True)
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            epochs_without_improvement = 0
            torch.save(model.state_dict(), checkpoint_path)
            epoch_pbar.write(f"✓ Epoch {epoch+1}: Saved best model with val_loss: {val_loss:.4f}")
        else:
            epochs_without_improvement += 1
        
        # Update progress bar
        epoch_pbar.set_postfix({
            'train_loss': f"{avg_train_loss:.4f}",
            'val_loss': f"{val_loss:.4f}",
            'val_acc': f"{val_accuracy:.4f}",
            'lr': f"{optimizer.param_groups[0]['lr']:.2e}"
        })
        
        # EarlyStopping check
        if epochs_without_improvement >= early_stopping_patience:
            epoch_pbar.write(f"\n⚠ Early stopping triggered after {epoch + 1} epochs")
            break
    
    # Load best model
    model.load_state_dict(torch.load(checkpoint_path))
    epoch_pbar.write(f"\n✓ Loaded best model from {checkpoint_path}")
    
    return results_dict

In [ ]:
dataloader = get_dataloader(DATASET, subset_ratio=0.1, batch_size=16)
x, y = next(iter(dataloader))
print(x.shape, y.shape)
del x, y

In [12]:
train_dataloader =  get_dataloader(train_dataset, subset_ratio=None, batch_size=16)
val_dataloader =  get_dataloader(val_dataset, subset_ratio=None, batch_size=16)
test_dataloader =  get_dataloader(test_dataset, subset_ratio=None, batch_size=16)

In [ ]:
results = training_loop(
    MODEL, 
    train_dataloader, 
    val_dataloader, 
    epochs=100, 
    learning_rate=2e-4,
    early_stopping_patience=10,              
    checkpoint_path='best_model_3DResNet.pth',  
    lr_patience=2
)

In [ ]:
model_save_path = './models/trained/3dResnet_Trained.pth'
torch.save(MODEL.state_dict(), model_save_path)

In [ ]:
results_df = pd.DataFrame.from_dict(results, orient="index").T
results_df.to_csv("3D_ResNet_Results.csv")

## Testing

In [19]:
MODEL.load_state_dict(torch.load("models\\trained_models\\3dResnet_Trained.pth", map_location=DEVICE))
MODEL.to(DEVICE)
MODEL.eval()
test_acc, test_loss = infer_loop(MODEL, test_dataloader)
test_acc, test_loss

(0.9375, 0.259842649102211)

# TensorFlow TinyVideoNet

In [4]:
import tensorflow as tf
import tensorflow.keras as k
import numpy as np
from models.TinyVideoNet import TinyVideoNetTransfer

c:\Users\Arnav Waghdhare\Desktop\Arnav20\Coding\Python\geture_research_ml\.venv\Lib\site-packages\tensorflow_hub\__init__.py:61: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import parse_version


In [5]:
def tf_data_generator(torch_dataset):
    def generator():
        for i in range(len(torch_dataset)):
            video, label, _ = torch_dataset[i]
            label = tf.one_hot(label, depth=8)
            yield video, label
    return generator

def create_dataset(torch_ds, batch_size=4):
    ds = tf.data.Dataset.from_generator(
        tf_data_generator(torch_ds),
        output_signature=(
            tf.TensorSpec(shape=(30, 3, 480, 640), dtype=tf.float32),
            tf.TensorSpec(shape=(8,), dtype=tf.float32)
        )
    )
    return ds.batch(batch_size)

In [ ]:
# Create TF Datasets
tf_train = create_dataset(train_dataset, batch_size=4)
tf_val = create_dataset(val_dataset, batch_size=4)
tf_test = create_dataset(test_dataset, batch_size=4)

In [7]:
model_handle = 'https://kaggle.com/models/google/tiny-video-net/frameworks/TensorFlow1/variations/tvn1/versions/1'
tf_model = TinyVideoNetTransfer(model_handle, num_classes=8)

In [8]:
class MetricsCallback(k.callbacks.Callback):
    def __init__(self):
        self.time_per_epoch = []  # Track epoch times
        self.epoch_start_time = None
        
    def on_epoch_begin(self, epoch, logs=None):
        self.epoch_start_time = time()
        
    def on_epoch_end(self, epoch, logs=None):
        # Calculate epoch time
        epoch_time = time() - self.epoch_start_time
        self.time_per_epoch.append(epoch_time)
        logs['time_per_epoch'] = epoch_time

# Create the callback
metrics_callback = MetricsCallback()

In [9]:
tf_model.compile(
    optimizer=k.optimizers.Adam(learning_rate=3e-4), 
    loss='categorical_crossentropy', 
    metrics=[
        'accuracy',
        k.metrics.Precision(),
        k.metrics.Recall(), 
        k.metrics.F1Score(average='macro')
    ])

## Training

In [ ]:
x, y = next(iter(tf_train))
print("x.shape:", x.shape)  # Should be (B, 30, 3, 480, 640)
print("y.shape:", y.shape)  # Should be (B,)
print("y.dtype:", y.dtype)  # Should be int32 or int64
print("Sample label:", y[0].numpy())  # Should be integer in [0, 7]

del x, y, DATASET, train_dataset, val_dataset, test_dataset, DEVICE
del torch, nn, tqdm, accuracy_score, precision_score, recall_score, f1_score
del DataLoader, Subset, random_split, ColorVideoDataset

x.shape: (8, 30, 3, 480, 640)
y.shape: (8, 8)
y.dtype: <dtype: 'float32'>
Sample label: [1. 0. 0. 0. 0. 0. 0. 0.]


In [ ]:
history = tf_model.fit(
    tf_train,
    validation_data=tf_val, 
    epochs=100, 
    callbacks=[
        tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10), 
        tf.keras.callbacks.ModelCheckpoint('./models/trained_models/TVN_best_model.weights.h5', monitor='val_loss', save_best_only=True, save_weights_only=True), 
        tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', patience=2, factor=0.9, min_lr=1e-7), 
        metrics_callback
        ]
    )

Epoch 1/100
      4/Unknown 13s 3s/step - accuracy: 0.1016 - f1_score: 0.0417 - loss: 3.7131 - precision: 0.1727 - recall: 0.1016

c:\Users\Arnav Waghdhare\Desktop\Arnav20\Coding\Python\geture_research_ml\.venv\Lib\site-packages\keras\src\trainers\epoch_iterator.py:164: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


4/4 ━━━━━━━━━━━━━━━━━━━━ 27s 8s/step - accuracy: 0.0938 - f1_score: 0.0357 - loss: 3.7818 - precision: 0.1500 - recall: 0.0938 - val_accuracy: 0.1562 - val_f1_score: 0.0338 - val_loss: 2.4679 - val_precision: 0.5000 - val_recall: 0.0312 - learning_rate: 3.0000e-04 - time_per_epoch: 27.0586
Epoch 2/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 14s 4s/step - accuracy: 0.2500 - f1_score: 0.1903 - loss: 2.9133 - precision: 0.3333 - recall: 0.1562 - val_accuracy: 0.0938 - val_f1_score: 0.0474 - val_loss: 2.3962 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - learning_rate: 3.0000e-04 - time_per_epoch: 13.7464
Epoch 3/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 12s 3s/step - accuracy: 0.0625 - f1_score: 0.0536 - loss: 3.0031 - precision: 0.0714 - recall: 0.0312 - val_accuracy: 0.0938 - val_f1_score: 0.0474 - val_loss: 2.3131 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - learning_rate: 3.0000e-04 - time_per_epoch: 12.4654
Epoch 4/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 13s 4s/step - accuracy: 0.1875 - f1_score: 0.1288 

KeyboardInterrupt: 

In [14]:
history.history

{'accuracy': [0.09375,
  0.15625,
  0.1875,
  0.09375,
  0.125,
  0.21875,
  0.1875,
  0.15625,
  0.28125,
  0.28125,
  0.25],
 'f1_score': [0.052083320915699005,
  0.07670453190803528,
  0.07131408900022507,
  0.05929486081004143,
  0.08571426570415497,
  0.16856059432029724,
  0.08012819290161133,
  0.0694444328546524,
  0.1409090757369995,
  0.1775640845298767,
  0.1577380746603012],
 'loss': [3.255795955657959,
  2.8296751976013184,
  2.4811947345733643,
  2.2535057067871094,
  2.7205843925476074,
  2.472210645675659,
  2.9941649436950684,
  2.7692158222198486,
  2.746459722518921,
  2.4065213203430176,
  2.6997034549713135],
 'precision': [0.04545454680919647,
  0.1818181872367859,
  0.1666666716337204,
  0.1428571492433548,
  0.1764705926179886,
  0.3571428656578064,
  0.1818181872367859,
  0.1111111119389534,
  0.2666666805744171,
  0.23529411852359772,
  0.29411765933036804],
 'recall': [0.03125,
  0.0625,
  0.0625,
  0.0625,
  0.09375,
  0.15625,
  0.125,
  0.0625,
  0.125,
  

In [15]:
results_df = pd.DataFrame.from_dict(history.history, orient="index").T
# results_df.to_csv("TVN_results_new.csv")

In [16]:
results_df

,accuracy,f1_score,loss,precision,recall,val_accuracy,val_f1_score,val_loss,val_precision,val_recall,learning_rate,time_per_epoch
0,0.09375,0.052083,3.255796,0.045455,0.03125,0.1250,0.063973,2.358474,0.0,0.0,3.000000e-04,30.477774
1,0.15625,0.076705,2.829675,0.181818,0.06250,0.0625,0.014706,2.520697,0.0,0.0,3.000000e-04,14.537724
2,0.18750,0.071314,2.481195,0.166667,0.06250,0.0625,0.014706,2.679544,0.0,0.0,3.000000e-04,13.722282
3,0.09375,0.059295,2.253506,0.142857,0.06250,0.0625,0.014706,2.686354,0.0,0.0,3.000000e-05,19.242203
4,0.12500,0.085714,2.720584,0.176471,0.09375,0.0625,0.014706,2.663268,0.0,0.0,3.000000e-05,26.440400
5,0.21875,0.168561,2.472211,0.357143,0.15625,0.0625,0.014706,2.659465,0.0,0.0,3.000000e-06,13.990036
6,0.18750,0.080128,2.994165,0.181818,0.12500,0.0625,0.014706,2.654320,0.0,0.0,3.000000e-06,15.134761
7,0.15625,0.069444,2.769216,0.111111,0.06250,0.0625,0.014706,2.653763,0.0,0.0,3.000000e-07,14.165493
8,0.28125,0.140909,2.746460,0.266667,0.12500,0.0625,0.014706,2.653128,0.0,0.0,3.000000e-07,14.139252
9,0.28125,0.177564,2.406521,0.235294,0.12500,0.0625,0.014706,2.652929,0.0,0.0,1.000000e-07,13.874323


## Testing

In [13]:
tf_model.build(input_shape=(None, 16, 3, 480, 640))

In [14]:
# Load the best model weights
tf_model.load_weights('models/trained_models/TVN_best_model.weights.h5')

# Evaluate the model on the test dataset
test_loss, test_accuracy = tf_model.evaluate(tf_test)

# Print the results
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")

c:\Users\Arnav Waghdhare\Desktop\Arnav20\Coding\Python\geture_research_ml\.venv\Lib\site-packages\keras\src\saving\saving_lib.py:797: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 10 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


InvalidArgumentError: Graph execution error:

Detected at node PyFunc defined at (most recent call last):
<stack traces unavailable>
TypeError: `generator` yielded an element of shape (16, 3, 480, 640) where an element of shape (30, 3, 480, 640) was expected.
Traceback (most recent call last):

  File "c:\Users\Arnav Waghdhare\Desktop\Arnav20\Coding\Python\geture_research_ml\.venv\Lib\site-packages\tensorflow\python\ops\script_ops.py", line 269, in __call__
    ret = func(*args)
          ^^^^^^^^^^^

  File "c:\Users\Arnav Waghdhare\Desktop\Arnav20\Coding\Python\geture_research_ml\.venv\Lib\site-packages\tensorflow\python\autograph\impl\api.py", line 643, in wrapper
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^

  File "c:\Users\Arnav Waghdhare\Desktop\Arnav20\Coding\Python\geture_research_ml\.venv\Lib\site-packages\tensorflow\python\data\ops\from_generator_op.py", line 235, in generator_py_func
    raise TypeError(

TypeError: `generator` yielded an element of sha

# PyTorch VideoMAE

In [4]:
from models.VideoMAE_SL import VideoMAEClassifier
from transformers import VideoMAEImageProcessor

c:\Users\Arnav Waghdhare\Desktop\Arnav20\Coding\Python\geture_research_ml\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
MODEL = VideoMAEClassifier(num_classes=8)

Loading weights: 100%|██████████| 184/184 [00:00<00:00, 368.99it/s, Materializing param=layernorm.weight]                                 
VideoMAEModel LOAD REPORT from: MCG-NJU/videomae-base
Key                                                                  | Status     |  | 
---------------------------------------------------------------------+------------+--+-
decoder.decoder_layers.{0, 1, 2, 3}.attention.attention.q_bias       | UNEXPECTED |  | 
decoder.decoder_layers.{0, 1, 2, 3}.attention.attention.v_bias       | UNEXPECTED |  | 
decoder.decoder_layers.{0, 1, 2, 3}.attention.attention.key.weight   | UNEXPECTED |  | 
decoder.decoder_layers.{0, 1, 2, 3}.attention.output.dense.bias      | UNEXPECTED |  | 
decoder.decoder_layers.{0, 1, 2, 3}.intermediate.dense.bias          | UNEXPECTED |  | 
decoder.decoder_layers.{0, 1, 2, 3}.layernorm_after.weight           | UNEXPECTED |  | 
decoder.decoder_layers.{0, 1, 2, 3}.attention.attention.value.weight | UNEXPECTED |  | 
decoder.decoder

## Dataloader and Training

In [13]:
def get_dataloader(dataset, subset_ratio : float | None = 0.1, batch_size : int = 2):
    processor = VideoMAEImageProcessor.from_pretrained("MCG-NJU/videomae-base")

    def collate_fn(batch):
        videos = []
        labels = []
        for video, label, _ in batch:
            video = video.permute(0, 2, 3, 1)
            video_list = [video[i].numpy() for i in range(video.shape[0])]

            video = processor(video_list, return_tensors="pt", do_normalize=False, do_rescale=False)["pixel_values"].squeeze(0) 
            # video = processor(video, return_tensors="pt", do_normalize=False, do_convert_rgb=False, do_center_crop=False)["pixel_values"].squeeze(0)
            videos.append(video)
            labels.append(torch.tensor(label, dtype=torch.long))
        
        videos = torch.stack(videos)
        labels = torch.stack(labels)
        return videos, labels
    
    if subset_ratio is not None:
        num_samples = int(len(dataset) * subset_ratio) 
        subset_indices = list(range(num_samples))
        subset = Subset(dataset, subset_indices)
        dataloader = DataLoader(subset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
    else:
        dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
    return dataloader

In [7]:
def training_loop(model, train_dataloader, val_dataloader = None, 
                  epochs=5, learning_rate=2e-4, early_stopping_patience=10,
                  checkpoint_path='best_model_VideoMAE.pth', lr_patience=2):

    results_dict = {
        'time_per_epoch': [],
        'train_accuracy': [],
        'train_loss': [],
        'val_accuracy': [],
        'val_loss': [],
        'precision': [],
        'recall': [],
        'f1': [],
        'learning_rate': [],
    }
    
    model = model.to(DEVICE)
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
    
    # ReduceLROnPlateau - tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', patience=2)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode='min', factor=0.1, patience=lr_patience, min_lr=1e-7
    )
    
    # EarlyStopping - tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)
    best_val_loss = float('inf')
    epochs_without_improvement = 0
    
    epoch_pbar = tqdm(range(epochs), desc="Training Epochs")

    for epoch in epoch_pbar:
        # Training phase
        model.train()
        running_loss = 0.0
        epoch_start = time()
        all_preds = []
        all_labels = []
        
        for videos, labels in train_dataloader:
            videos, labels = videos.to(DEVICE), labels.to(DEVICE)

            optimizer.zero_grad()
            outputs = model(videos)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            all_preds.extend(outputs.argmax(dim=1).cpu().detach().numpy())
            all_labels.extend(labels.cpu().detach().numpy())
        
        epoch_time = time() - epoch_start
        results_dict['time_per_epoch'].append(epoch_time)
        
        # MetricsCallback - tracks precision, recall, f1
        results_dict['train_accuracy'].append(accuracy_score(all_labels, all_preds))
        results_dict['precision'].append(precision_score(all_labels, all_preds, average='weighted', zero_division=0))
        results_dict['recall'].append(recall_score(all_labels, all_preds, average='weighted', zero_division=0))
        results_dict['f1'].append(f1_score(all_labels, all_preds, average='weighted', zero_division=0))
        
        avg_train_loss = running_loss / len(train_dataloader)
        results_dict['train_loss'].append(avg_train_loss)
        
        if val_dataloader is not None:
            val_accuracy, val_loss = infer_loop(model, val_dataloader, criterion)
        
            results_dict['val_accuracy'].append(val_accuracy)
            results_dict['val_loss'].append(val_loss)
            results_dict['learning_rate'].append(optimizer.param_groups[0]['lr'])
            
            # ReduceLROnPlateau callback
            scheduler.step(val_loss)
            
            # ModelCheckpoint - tf.keras.callbacks.ModelCheckpoint('best_model_TVN.h5', monitor='val_loss', save_best_only=True)
            if val_loss < best_val_loss:
                best_val_loss = val_loss
                epochs_without_improvement = 0
                torch.save(model.state_dict(), checkpoint_path)
                epoch_pbar.write(f"✓ Epoch {epoch+1}: Saved best model with val_loss: {val_loss:.4f}")
            else:
                epochs_without_improvement += 1
            
            # Update progress bar
            epoch_pbar.set_postfix({
                'train_loss': f"{avg_train_loss:.4f}",
                'val_loss': f"{val_loss:.4f}",
                'val_acc': f"{val_accuracy:.4f}",
                'lr': f"{optimizer.param_groups[0]['lr']:.2e}"
            })
        else:
            # Update progress bar without validation
            epoch_pbar.set_postfix({
                'val_loss' : "Not Given",
                'train_loss': f"{avg_train_loss:.4f}",
                'lr': f"{optimizer.param_groups[0]['lr']:.2e}"
            })        
        # EarlyStopping check
        if epochs_without_improvement >= early_stopping_patience:
            epoch_pbar.write(f"\n⚠ Early stopping triggered after {epoch + 1} epochs")
            break
    
    return results_dict

In [8]:
def infer_loop(model, dataloader, criterion = None) -> tuple:
    if criterion is None:
        criterion = nn.CrossEntropyLoss()
    
    running_loss = 0.0
    all_preds = []
    all_labels = []
    model.eval()
    for videos, labels in dataloader:
        videos, labels = videos.to(DEVICE), labels.to(DEVICE)

        outputs = model(videos)
        loss = criterion(outputs, labels)
        running_loss += loss.item()
        all_preds.extend(outputs.argmax(dim=1).cpu().detach().numpy())
        all_labels.extend(labels.cpu().detach().numpy())
        
    val_accuracy = accuracy_score(all_preds, all_labels)
    avg_loss = running_loss / len(dataloader)

    return (val_accuracy, avg_loss)

In [14]:
train_dataloader =  get_dataloader(train_dataset, subset_ratio=0.1, batch_size=16)
val_dataloader =  get_dataloader(val_dataset, subset_ratio=0.1, batch_size=16)
test_dataloader =  get_dataloader(test_dataset, subset_ratio=None, batch_size=16)

In [15]:
results = training_loop(
    MODEL, 
    train_dataloader, val_dataloader
)

Training Epochs:  20%|██        | 1/5 [00:15<01:00, 15.24s/it, train_loss=2.1107, val_loss=2.4723, val_acc=0.0000, lr=2.00e-04]

✓ Epoch 1: Saved best model with val_loss: 2.4723


Training Epochs:  40%|████      | 2/5 [00:27<00:40, 13.58s/it, train_loss=2.1546, val_loss=2.4260, val_acc=0.0000, lr=2.00e-04]

✓ Epoch 2: Saved best model with val_loss: 2.4260


Training Epochs:  60%|██████    | 3/5 [00:40<00:26, 13.10s/it, train_loss=2.0770, val_loss=2.3822, val_acc=0.0000, lr=2.00e-04]

✓ Epoch 3: Saved best model with val_loss: 2.3822


Training Epochs:  80%|████████  | 4/5 [00:52<00:12, 12.92s/it, train_loss=2.1402, val_loss=2.3460, val_acc=0.0000, lr=2.00e-04]

✓ Epoch 4: Saved best model with val_loss: 2.3460


Training Epochs: 100%|██████████| 5/5 [01:05<00:00, 13.01s/it, train_loss=2.1253, val_loss=2.3057, val_acc=0.0000, lr=2.00e-04]

✓ Epoch 5: Saved best model with val_loss: 2.3057


In [ ]:
model_save_path = './models/trained_models/VideoMAE_Trained.pth'
torch.save(MODEL.state_dict(), model_save_path)

In [43]:
results_df = pd.DataFrame.from_dict(results, orient="index").T
results_df.to_csv("VideoMAE_Results.csv")

In [44]:
results_df

,time_per_batch,time_per_epoch,train_accuracy,train_loss,val_accuracy,val_loss,precision,recall,f1,learning_rate
0,NaN,10.044201,0.04,2.230697,NaN,NaN,0.0016,0.04,0.003077,NaN
1,NaN,9.433681,0.16,2.254749,NaN,NaN,0.0256,0.16,0.044138,NaN
2,NaN,9.365645,0.16,2.132989,NaN,NaN,0.0256,0.16,0.044138,NaN
3,NaN,9.248171,0.16,2.123748,NaN,NaN,0.0256,0.16,0.044138,NaN
4,NaN,9.534895,0.16,2.102612,NaN,NaN,0.0256,0.16,0.044138,NaN


## Testing

In [ ]:
MODEL.load_state_dict(torch.load("models\\trained_models\\VideoMAE_Trained.pth", map_location=DEVICE))
MODEL.to(DEVICE)
MODEL.eval()
test_acc, test_loss = infer_loop(MODEL, test_dataloader)
test_acc, test_loss